# Assignment 6: Appearance-based gaze estimation - Task 03

Author: `Yintao, Xu` | Date: `2020-02-15` | Email: xuyt@shanghaitech.edu.cn

Topics: `numpy`

**Note**: Make sure you have completed task 01 & 02.

HOG, as a feature engineering method, could improve your estimation performance.

![](figures/HOG_visual.png)

In [ ]:
# IMPORTANT: run this cell before runnng any cell to activate auto re-import
%load_ext autoreload
%autoreload 2

# 'gazelib' is the toolkit provided by this assignment, at the same directory of this notebook
from gazelib.utils import *
import numpy as np
import time
import matplotlib.pyplot as plt
import random

In [ ]:
# Datasets has been supposed to be downloaded at last notebook
# You could run thie cell to have a check :)
download_csv_mpIIdataset()

In [ ]:
# load the training dataset
train_df = load_train_csv_as_df()
train_df.head(10)

In [ ]:
# transform data into numpy arrays
def df2nptensor(df):
    imgs = []
    imgs_HOG = []
    gaze_dirs = []

    print_interval = 1000
    print_cnter = 0
    
    for _, i in df.iterrows():
        if print_cnter % print_interval == 0:
            print("[{} / {}]".format(print_cnter, len(df)), end='\r')
        print_cnter += 1
        im_arr = decode_base64_img(i['image_base64'])
        gaze_dirs.append([i['yaw'], i['pitch']])
        im = im_arr / 255
        
        imgs.append(im)
    
    gaze_dirs = np.array(gaze_dirs)
    imgs = np.array(imgs)
    
    return gaze_dirs, imgs

# For effciency, we only takes first 5,000 samples. Pick subject 5 as validation 
# set and the rest of the dataset as training set
SAMPLE_NUM = 5000
print("Start to generate sampled dataset, it may take ~10s.")
train_Y, train_X = df2nptensor(train_df[train_df["subject_id"] != 5][: int(SAMPLE_NUM * 0.8)])
val_Y, val_X = df2nptensor(train_df[train_df["subject_id"] == 5][: int(SAMPLE_NUM * 0.2)])

print("train_X.shape: {}".format(train_X.shape))
print("train_Y.shape: {}".format(train_Y.shape))
print("val_X.shape: {}".format(val_X.shape))
print("val_X.shape: {}".format(val_Y.shape))

## Roadmap

At previous sections, estimation pipeline for gaze vector has been built:

<img src="figures/gaze_model_pipeline.png" style="zoom:80%" />

Formalize the input/ouput of our estimation pipeline mathematically:
- `Input`: the gray-scale image $I\in \mathbb{R}^{18\times30}$ (18x30 numpy array)
- `Output`: two floats: yaw($\gamma \in \mathbb{R}$), pitch($\theta  \in \mathbb{R}$)


In previous sections, we have evaluated the simlarity among images by Euclidean distance. However, naive computation over raw image is computationally expensive.

## Section 3.0: Motivation: Why HOG? 


We have 42,000 images in our pandas data frame. Assume in each estimation, we require doing one float number multplication for each pixels in every image.

In [ ]:
a = np.random.randn(42000 * 18 * 30,)
b = np.random.randn(42000 * 18 * 30,)

time_start = time.time()
c = a * b
time_end = time.time()

print("Multiply 42000 * 18 * 30 numbers cost: {:.4f}s".format(time_end - time_start))

**Large computational cost**: It runs ~0.06 seconds on *Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz*, which sounds tractable. However, you may repeat 10,000 ~ 20,000 such operation, which means roughly that it will cost ~ 20 min. Besides, memory cost is also considerable. Therefore, we require a **dimension reduction** method to reduce the computation. 

**Not robust to translation**: If we apply Euclidean distance to images, when a slight translation of image occurs, it may lead to dramatic change to output.

In computer vision community, **HOG(histogram of oriented gradient)** is a frequently used method to do the feature engineering of images with tractable size of dimension and much robustness.

## Section 3.1: Gradient of Image [2]

There is a widely accepted assumption in computer vision community: 
> local object appearance and shape can often be characterized rather well by **the distribution of local intensity gradients or edge directions**, even without precise knowledge of the corresponding gradient or edge posistions.

Therefore, studying the **image gradient**[3] is important. It is a directional change in the intensity or color in an image. 

![](figures/HOG.png)

Mathematically, the gradient of a two-variables function (here the image intensity function) at each image point is a 2D vector with the components given by the derivatives in the horizontal and vertical directions.

An image in modern devices can be viewed as a matrix of pixel intensity, which is discrete. However, derivatives of this function cannot be defined unless we assume that there is an underlying continuous intensity function which has been sampled at the image points. 

The most common way to approximate the image gradient is to **convolve an image with a kernel**, such as the [**Sobel operator**](https://en.wikipedia.org/wiki/Sobel_operator).

Let's first start with **2D convolution**.

### Section 3.1.1: 2D Convolution[1] (15%)
Convolution is an important class of operators in linear time-invariant (LTI) system. You start with a **kernel**, which is simply a small matrix of weights. This kernel “slides” over the 2D input data, performing an element-wise multiplication with the part of the input it is currently on, and then summing up the results into a single output pixel.

![ChessUrl](http://data.liubai01.cn:81/f/c518901817914d7f815e/?dl=1 "chess") 

Formally, it is denoted as $*$:

$$
    \textbf{O} = \textbf{F} * \textbf{I}
$$

with $\textbf{O}$ corresponding to output, $\textbf{F}\in \mathbb{R}^{2K+1\times2K+1}$ corresponding to filter kernel, $\textbf{I}\in \mathbb{R}^{W \times H}$ corresponding to the input Image.

The formula can be written as follows:

$$
    \textbf{O}_{i,j} = \sum_{k=0}^{2K} \sum_{l=0}^{2K} \textbf{F}_{k,l} \times \textbf{I}_{k + i, l + j}, i \in [0, W -2], j \in [0, H - 2]
$$

For instance, for the upper-left corner of output value 12 at previous figure:

$$
    12 = 3 \times 0 + 3 \times 1 + 2 \times 2 + 0 \times 2 + 0 \times 2 + 1 \times 0 + 3 \times 0 + 1 \times 1 + 2 \times 2
$$

*Remark: For those who has background in signal & system, note the inconsistent meaning of "Conovolution" in mathematical(signal&system) context and machine learning context. In most cases, convolution in machine learning is the correlation in signal & system, which means you are not required to flip the kernel. It was a historical misnormer here, but is accepted by most scientists nowadays.*

![](figures/code_time.png)
**Complete code at `AppearanceGazeEst/conv2d3x3`(15%)**

In [ ]:
# Local Test - Section 3.1.1 (15%)
# Note: feel free to print out your result to debug if it cannot pass assert_eq_np
from AppearanceGazeEst import conv2d3x3
from gazelib.task_2_judge import assert_eq_np

img = np.array([
    [3, 3, 2, 1, 0],
    [0, 0, 1, 3, 1],
    [3, 1, 2, 2, 3],
    [2, 0, 0, 2, 2],
    [2, 0, 0, 0, 1]
])
kernel = np.array([
    [0, 1, 2],
    [2, 2, 0],
    [0, 1, 2]
])
gt_out = np.array([
    [12, 12, 17],
    [10, 17, 19],
    [9, 6, 14]
])

out = conv2d3x3(img, kernel)
assert_eq_np(gt_out, out)

print("Pass Section 3.1.1 - convolution 2d local test(15%).")

### Section 3.1.2: Compute Gradient By Sobel Operator (15%) - Apply convolution by numpy [4]
The Sobel operator is a discrete differentiation operator, computing an approximation of the gradient of the image intensity function.

The operator uses two 3×3 kernels which are convolved with the original image to calculate approximations of the derivatives – one for horizontal changes, and one for vertical. The 2D convolution has been implemented by previous sections. If we define I as the source image, and Gx and Gy are two images which at each point contain the vertical and horizontal derivative approximations respectively, the computations are as follows:

$$
\mathbf {G} _{x}={\begin{bmatrix}-1&0&1\\-2&0&2\\-1&0&1\end{bmatrix}}*\mathbf {I} \quad {\mbox{and}}\quad \mathbf {G} _{y}={\begin{bmatrix}-1&-2&-1\\0&0&0\\1&2&1\end{bmatrix}}*\mathbf {I}
$$

In most implementations, they also apply gaussian kernel to smooth the image before getting gradients.

![](figures/eye_grad_new.png)

Formally, for each pixel
$$
Mag = \sqrt{g _{x}^2 + g_{y}^2}
$$
$$
\mathbf{Dir} = [g _{x}; g_{y}]
$$
$$
\mathbf{Dir}_{norm} = \mathbf{Dir} / (Mag + 10^{-3})
$$
Here, 10e-3 is applied to improve numerical robustness in case of zero magnitude.

![](figures/code_time.png)
**Complete the code of the sobel gradient computation at `AppearanceGazeEst.py/compute_grad`(15%)**

In [ ]:
# Local Test - Section 3.1.2 (15%)
# Note: feel free to print out your result to debug at gazelib.task_2_judge.py/assert_eq_2223
# if it cannot pass assert_eq_np
from AppearanceGazeEst import compute_grad
from gazelib.task_2_judge import assert_eq_2223

sanity_im = np.array([
    [1., 1., 7.], 
    [2., 1., 8.], 
    [3., 5., 2.]
])

grad_dir, grad_mag = compute_grad(sanity_im)
assert_eq_2223(grad_dir, grad_mag)
print("You pass the local test - Section 3.1.2 (15%)")

In [ ]:
# sample an image to visualize the gradient computation
sample_im = train_X[random.randint(0, train_X.shape[0])]
gaze_dir, gaze_mag = compute_grad(sample_im)
vis.vis_grad(sample_im, gaze_dir[0] * gaze_mag, gaze_dir[1] * gaze_mag)

## Section 3.2 Histogram of oriented gradient (10%, Bonus 10%)
Recall in section 2.3:
> local object appearance and shape can often be characterized rather well by **the distribution of local intensity gradients or edge directions**, even without precise knowledge of the corresponding gradient or edge posistions.

When finishing computing of gradients, at next step, you should get the distribution over an image (or an image patch). Using histogram is an intuitive way to model a distribution with many weighted data points(gradients of pixels, with direction and magnitude). This step is called **Orientation Binning**.

### Section 3.2.1: Details of Orientation Binning (Optional Reading)

**Note**: You are not required to go to very details of Orientation Binning. We'll provide you with a reference code.

<img src="figures/hist_grad_vis.png_fix.png" style="zoom:60%" />

**Range of angle**[6] In this assignment, angles are derived from `np.arctan2` towards gradient direction(x1,x2), which means that it ranges from $[-\pi, +\pi]$.

![](figures/arctan2_wiki.png)

**Bilinear voting policy**[5] A bin is selected based on the direction, and the vote ( the value that goes into the bin ) is selected based on the magnitude. 

Let’s first focus on the pixel encircled in blue. It has an angle ( direction ) of 30 degrees and magnitude of 5. So it adds 5 to the 30-deg. bin. The gradient at the pixel encircled using red has an angle of -40 degrees and magnitude of 3. Since -40 degrees is between -60 and -30, the vote by the pixel splits into the two bins 
proportional to 1/dist. to the bin.

**Boundary condtion** When angle goes to boundary, for example, 160 degrees, it should be going to bin of 150-deg and -180-deg. Due to the range of `np.arctan2`, angle skips numerically at 180 and -180 but is adajacent at Cartesian coordination.

### Section 3.2.2: Numpy: vectorization [7]

Vectorization is used to speed up the Python code without using loop. Numpy will invokes C-code directly at backend, which is more rapid than python loops.

In [ ]:
# Try this cell to see the power of vectorization
a = np.random.randn(100000,)
b = np.random.randn(100000,)

c = 0
# python loops
time_start = time.time()
for i in range(100000):
    c = a[i] + b[i]
time_nonvec = time.time() - time_start

# vectorization
time_start = time.time()
c_ = np.sum(a + b)
time_vec = time.time() - time_start

speed_up_per = (time_nonvec / time_vec - 1) * 100

print("Before vectorization: {:.3f} s".format(time_nonvec))
print("After vectorization: {:.3f} s".format(time_vec))
print("Speedup: {:.2f}%".format(speed_up_per))

### Section 3.2.3: Re-implement HOG - learn vectorization at numpy(Bonus, 10%)

**Note**: the non-vectorized version is at `ApperanceGazeEstV2/bilinear_HOG_nonvec` above the target function `bilinear_HOG`. Please gaurantee that your implementation shares the same output as it and speeds up at least 200%.

![](figures/code_time.png)
**Complete the code of the HOG computation at `AppearanceGazeEst.py/bilinear_HOG`(10%)**

In [ ]:
# Local Test - Section 3.2.3 (Bonus, 10%)
# Note: feel free to print out your result to debug at gazelib.task_2_judge.py/assert_eq_2223
# if it cannot pass assert_eq_np
from gazelib.HOG import bilinear_HOG_patch_nonvec
from AppearanceGazeEst import bilinear_HOG
import time

sample_im = train_X[5]
grad_dir, grad_mag = compute_grad(sample_im)
time_nonvec = 0
time_vec = 0
exp_num = 6

for _ in range(exp_num): # repeat experiments due to randomness of OS scheduling
    time_start = time.time()
    a = bilinear_HOG_patch_nonvec(gaze_dir, gaze_mag)
    time_nonvec += time.time() - time_start

    time_start = time.time()
    b = bilinear_HOG(gaze_dir, gaze_mag)
    time_vec += time.time() - time_start

speed_up_per = (time_nonvec / time_vec - 1) * 100
assert_eq_np(a, b)
assert speed_up_per > 200

print("Before vectorization(avg): {:.3f} s".format(time_nonvec / exp_num))
print("After vectorization(avg): {:.3f} s".format(time_vec / exp_num))
print("Speedup: {:.2f}%".format(speed_up_per))
print("You pass the local test - Section 3.2.3 (Bonus, 10%)")

In [ ]:
# visualize a random image
random_idx = random.randint(0, train_X.shape[0])
sample_im = train_X[random_idx]
grad_dir, grad_mag = compute_grad(sample_im)

hist = bilinear_HOG(grad_dir, grad_mag)

vis.vis_HOG(sample_im, grad_dir[0], grad_dir[1], hist)

### Section 3.2.4: Descriptor blocks
To keep spatial information, in practice, HOG splits image into blocks to get histograms, then concatenates histograms in a fixed order(e.g: row major) to a vector.

<img src="figures/desc_block.png" style="zoom:60%" />

**Note**: In this section, we give the implementation of descriptor blocks based on your previous function of HOG. If you do not complete the bonus(section 3.2.3), modify `bilinear_HOG_DB` function maually to use non-vectorization version of HOG. It uses the vectorization version by default.

In [ ]:
from AppearanceGazeEst import bilinear_HOG_DB

random_idx = random.randint(0, train_X.shape[0])
sample_im = train_X[random_idx]
grad_dir, grad_mag = compute_grad(sample_im)

# You can modify hyperparameter patch_num here, though not recommended
patch_num = (3, 4)
hist = bilinear_HOG_DB(sample_im, patch_num=patch_num)

vis.vis_HOG_full(sample_im, grad_dir[0], grad_dir[1], hist, patch_num=patch_num)

## Section 3.3 Combine HOG with K-NN (5%)

Recall at previous sections, we applied HOG to raw image to transform it into a vector. Let's apply HOG(completed on previous notebook) to all images here.

Then, we could do KNN over the HOGized images. You would see the improvement of performance with respect to the previous sections. Since our python-based implementation of HOG is inefficient, the estimation process may be a little bit long. 
![](figures/code_time.png)
**Complete the code of the HOG computation at `ApperanceGazeEstV2/KNN_HOG`(5%)**

In [ ]:
# Local Test - Eye images
# Note: feel free to print out your result to debug if it cannot pass assert_eq_np
from AppearanceGazeEst import KNN_HOG
from gazelib.task_2_judge import assert_eq_np
from gazelib.task_3_judge import compute_angle_error

idx = 10
# truncate the traininig set to improve the performance
ret = KNN_HOG(train_X, train_Y, val_X[idx], 5)
print(ret)
assert_eq_np(ret, np.array([0.10192861, -0.06430973]))

print("Pass local test@3.3 - eye images")
plt.figure(figsize=(4, 3))
vis.visualize_est(val_X[idx], ret, val_Y[idx])
plt.title("Angle Error: {:.3f}".format(compute_angle_error(val_Y[idx], ret)));

![](figures/good_job_banner.png)
You should have completed **all cells(35% + 10% Bonus)** in this task locally when you reach here!

**CheckList**

- conv2d (15%)
- compute_grad (15%)
- bilinear_HOG (Bonus, 10%)
- KNN_HOG (5%)

## Reference

- [1] Intuitively Understanding Convolutions for Deep Learning: https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1
- [2] Image gradient wiki: https://en.wikipedia.org/wiki/Image_gradient
- [3] One of image gradient visualization is imported from: https://www.learnopencv.com/histogram-of-oriented-gradients/
- [4] https://en.wikipedia.org/wiki/Sobel_operator
- [5] https://www.learnopencv.com/histogram-of-oriented-gradients/
- [6] https://en.wikipedia.org/wiki/Atan2
- [7] https://www.geeksforgeeks.org/vectorization-in-python/